# Neuralmt Documentation

In [1]:
from default import *
import os, sys

## Run the baseline solution on dev

In [2]:
model = Seq2Seq(build=False)
model.load(os.path.join('data', 'seq2seq_E049.pt'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# loading test dataset
test_iter = loadTestData(os.path.join('data', 'input', 'dev.txt'), model.fields['src'],
                            device=device, linesToLoad=sys.maxsize)
results = translate(model, test_iter) # Warning: will take >5mins depending on your machine
print("\n".join(results))

1305it [06:55,  3.14it/s]

when when i was in my 20s , i had my first <unk> . . 
i was a and i was particularly in berkeley . berkeley . 
she was a a woman woman named alex . 
when the came came up the first first , she came up and she a a a , and she fell into the couch in my office , <unk> her her and and told me , about about about . 
and when i heard this , i was was . 
my my , , had a a patient patient for a first patient . 
and i got a woman in the people who wanted to talk about about . 
i 'll leave that , i thought , . 
but i did n't have it . 
with the the stories , that alex opened into the , was easy easy to me just to with with head , while we were the problems . 
" 30 is the new 20 " , " alex , and as far as i did , i had right . 
work came later later , later came later later , later came later later later later came later later . 
people people in the the , and alex , and i did n't have time . 
but soon my my my was , my head , to ask . 
i i was going to . 
i said , " yeah , you know , you 're wit

## Evaluate the baseline output

In [3]:
from bleu_check import bleu
ref_t = []
with open(os.path.join('data','reference','dev.out')) as r:
    ref_t = r.read().strip().splitlines()
print(bleu(ref_t, results))

BLEU = 17.11 54.0/24.1/12.0/6.5 (BP = 0.957 ratio = 0.958 hyp_len = 23858 ref_len = 24902)


# Class AttentionModule

In [4]:
class AttentionModule(nn.Module):
    def __init__(self, attention_dim):
        super(AttentionModule, self).__init__()
        self.W_enc = nn.Linear(attention_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(attention_dim, attention_dim, bias=False)
        self.V_att = nn.Linear(attention_dim, 1, bias=False)
        return

    # Start working from here, both 'calcAlpha' and 'forward' need to be fixed
    def calcAlpha(self, decoder_hidden, encoder_out):
        seq, batch, dim = encoder_out.shape
        score_i = torch.add(self.W_enc(encoder_out), self.W_dec(decoder_hidden))
        alpha = torch.nn.functional.softmax(self.V_att(torch.tanh(score_i)))
        alpha = alpha.permute(1,0,2)

        return alpha

    def forward(self, decoder_hidden, encoder_out):
        alpha = self.calcAlpha(decoder_hidden, encoder_out)
        seq, _, dim = encoder_out.shape
        product = torch.mul(alpha.permute(1,0,2), encoder_out)
        context = (torch.sum(product, dim=0)).reshape(1, 1, dim)
        return context, alpha.permute(2, 0, 1)

For the calcAlpha function, we first calculate the score using torch.add() function to add W_enc(h_enc) and W_dec(h_dec) together. Then we use softmax() and tanh() from pytorch to obtain the result of softmax(V_att(tanh(score))) which is the value of alpha. In order to maintain alpha's correct shape as the one in default, we use alpha.permute(1,0,2).

In the forward function, alpha is calculated by using calcAlpha(). Then we use multiply function from torch to multiply alpha_i and hi_enc (permute the alpha in order to successfully multiply it with encoder_out). Then we use torch.sum to sum the product of alpha and encoder_out with dim = 0 (normalizes values along axis 0) and reshape it to obtain same correct shape as default solution. Finally we return the context vector with permuted alpha (match alpha's shape in default solution).

# GreedyDecoder function

In [5]:
def greedyDecoder(decoder, encoder_out, encoder_hidden, maxLen,
                  eos_index):
    seq1_len, batch_size, _ = encoder_out.size()
    target_vocab_size = decoder.target_vocab_size

    outputs = torch.autograd.Variable(
        encoder_out.data.new(maxLen, batch_size, target_vocab_size))
    alphas = torch.zeros(maxLen, batch_size, seq1_len)
    # take what we need from encoder
    decoder_hidden = encoder_hidden[-decoder.n_layers:]
    # start token (ugly hack)
    output = torch.autograd.Variable(
        outputs.data.new(1, batch_size).fill_(eos_index).long())
    for t in range(maxLen):
        output, decoder_hidden, alpha = decoder(
            output, encoder_out, decoder_hidden)
        outputs[t] = output
        alphas[t] = alpha.data
        output = torch.autograd.Variable(output.data.max(dim=2)[1])
        if int(output.data) == eos_index:
            break
    return outputs, alphas.permute(1, 2, 0)



# Translate function

In [6]:
def translate(model, test_iter):
    results = []
    for i, batch in tqdm(enumerate(test_iter)):
        output, attention = model(batch.src)
        output = output.topk(1)[1]
        output = model.tgt2txt(output[:, 0].data).strip().split('<EOS>')[0]
        results.append(output)
    return results

## Analysis

We implement the default.py according to the Baseline. For calcAlpha function in class AttentionModule, we define score_i to save the output of adding W_enc(h_enc) and W_dec(h_dec). We then using softmax function to generate alpha. For foward function in class AttentionModule, we multiply the alpha and h_enc and sum the product together to get context vector. Eventually, we improve the score from 3.35 to 17.11.